In [2]:
#based on http://www.jw.pe/blog/post/quantifying-sufjan-stevens-with-the-genius-api-and-nltk/
import requests
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from IPython.display import clear_output

CLIENT_ACCESS_TOKEN = "lTv0ZCyw1mkXQ7_9bZ9GmEdc9vPh2_VQzhFCpERSOYiFA1fXjEVrI8BY-D9_k6E2"
BASE_URI = "https://api.genius.com"

In [4]:
#from: https://gist.github.com/imdkm/a60247b59ff1881fa4bb8846a9b44c96
# send request and get response in json format.
def _get(path, params=None, headers=None):

    # generate request URL
    requrl = '/'.join([BASE_URI, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

print("searching Arjona's artist id. \n")

# find artist id from given data.
find_id = _get("search", {'q': 'Ricardo Arjona'})
for hit in find_id["response"]["hits"]:
   if hit["result"]["primary_artist"]["name"] == 'Ricardo Arjona':
       artist_id = hit["result"]["primary_artist"]["id"]
       break

print("-> Ricardo Arjona's id is " + str(artist_id) + "\n")

searching Arjona's artist id. 

-> Ricardo Arjona's id is 358670



In [5]:
ARJONA_ARTIST_ID = 358670

In [7]:
def _get(path, params=None, headers=None):

    url = '/'.join([BASE_URI, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}
    response = requests.get(url=url, params=params, headers=headers)
    response.raise_for_status()

    return response.json()


def get_artist_songs(artist_id):

    current_page = 1
    next_page = True
    songs = []

    while next_page:
        path = "artists/{}/songs/".format(artist_id)
        params = {'page': current_page}
        data = _get(path=path, params=params)
        page_songs = data['response']['songs']
        if page_songs:
            songs += page_songs
            current_page += 1
        else:
            next_page = False

    return songs


In [8]:
songs = get_artist_songs(ARJONA_ARTIST_ID)
songs = [song for song in songs if song['primary_artist']['id'] == ARJONA_ARTIST_ID]

songs

[{'annotation_count': 1,
  'api_path': '/songs/1833971',
  'full_title': 'Abarroteria de amor by\xa0Ricardo\xa0Arjona',
  'header_image_thumbnail_url': 'https://images.genius.com/12b6da30eea50fdbcd886f8793409886.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/12b6da30eea50fdbcd886f8793409886.1000x1000x1.jpg',
  'id': 1833971,
  'lyrics_owner_id': 1549345,
  'lyrics_state': 'complete',
  'path': '/Ricardo-arjona-abarroteria-de-amor-lyrics',
  'primary_artist': {'api_path': '/artists/358670',
   'header_image_url': 'https://images.genius.com/12b6da30eea50fdbcd886f8793409886.1000x1000x1.jpg',
   'id': 358670,
   'image_url': 'https://images.genius.com/12b6da30eea50fdbcd886f8793409886.1000x1000x1.jpg',
   'is_meme_verified': False,
   'is_verified': False,
   'name': 'Ricardo Arjona',
   'url': 'https://genius.com/artists/Ricardo-arjona'},
  'pyongs_count': None,
  'song_art_image_thumbnail_url': 'https://images.genius.com/12b6da30eea50fdbcd886f8793409886.300x300x1.jpg',
 

In [15]:
def scrape_lyrics(url):

    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    lyrics = soup.find("div", {"class": "lyrics"})
    lyrics = cleanhtml(str(lyrics))
    return lyrics

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

total = len(songs)
for i, song in enumerate(songs):
    if 0 == i % 10:
        clear_output(wait=True)
        print("%d songs parsed out of %d" % (i, total))
    url = song['url']
    lyrics = scrape_lyrics(url)
    song['lyrics'] = lyrics

250 songs parsed out of 256


In [16]:
df = pd.read_json(json.dumps(songs))
df.to_csv('data/arjona_raw_songs.csv', sep=';',index=False)

In [38]:
#df = pd.read_csv('data/arjona_raw_songs.csv', sep=';')

### Let's clean the db a little bit

In [39]:
df.head()

,annotation_count,api_path,full_title,header_image_thumbnail_url,header_image_url,id,lyrics,lyrics_owner_id,lyrics_state,path,primary_artist,pyongs_count,song_art_image_thumbnail_url,stats,title,title_with_featured,url
0,1,/songs/1833971,Abarroteria de amor by Ricardo Arjona,https://images.genius.com/12b6da30eea50fdbcd88...,https://images.genius.com/12b6da30eea50fdbcd88...,1833971,\n\nEl misterio estaba allí\nEn aquella casa c...,1549345,complete,/Ricardo-arjona-abarroteria-de-amor-lyrics,"{'api_path': '/artists/358670', 'header_image_...",NaN,https://images.genius.com/12b6da30eea50fdbcd88...,"{'hot': False, 'unreviewed_annotations': 0}",Abarroteria de amor,Abarroteria de amor,https://genius.com/Ricardo-arjona-abarroteria-...
1,1,/songs/985362,A cara o cruz by Ricardo Arjona,https://images.genius.com/12b6da30eea50fdbcd88...,https://images.genius.com/12b6da30eea50fdbcd88...,985362,\n\nEl azar es la metáfora perfecta\nDe quien ...,1549345,complete,/Ricardo-arjona-a-cara-o-cruz-lyrics,"{'api_path': '/artists/358670', 'header_image_...",NaN,https://images.genius.com/12b6da30eea50fdbcd88...,"{'hot': False, 'unreviewed_annotations': 0}",A cara o cruz,A cara o cruz,https://genius.com/Ricardo-arjona-a-cara-o-cru...
2,1,/songs/1141487,Acompáñame a Estar Solo by Ricardo Arjona,https://images.genius.com/f4d3becb5b5170db9345...,https://images.genius.com/f4d3becb5b5170db9345...,1141487,\n\nAcompáñame a estar solo\nA purgarme los fa...,1549345,complete,/Ricardo-arjona-acompaname-a-estar-solo-lyrics,"{'api_path': '/artists/358670', 'header_image_...",NaN,https://images.genius.com/f4d3becb5b5170db9345...,"{'hot': False, 'unreviewed_annotations': 0}",Acompáñame a Estar Solo,Acompáñame a Estar Solo,https://genius.com/Ricardo-arjona-acompaname-a...
3,1,/songs/2123441,Acompñame a estar solo - en vivo by Ricardo Ar...,https://images.genius.com/12b6da30eea50fdbcd88...,https://images.genius.com/12b6da30eea50fdbcd88...,2123441,\n\nAcompáñame a estar solo\nA purgarme los fa...,1549345,complete,/Ricardo-arjona-acompname-a-estar-solo-en-vivo...,"{'api_path': '/artists/358670', 'header_image_...",NaN,https://images.genius.com/12b6da30eea50fdbcd88...,"{'hot': False, 'unreviewed_annotations': 0}",Acompñame a estar solo - en vivo,Acompñame a estar solo - en vivo,https://genius.com/Ricardo-arjona-acompname-a-...
4,1,/songs/1567279,Adios Melancolia by Ricardo Arjona,https://images.genius.com/12b6da30eea50fdbcd88...,https://images.genius.com/12b6da30eea50fdbcd88...,1567279,"\n\nTengo un domingo en stand by, por si algÃº...",1549345,complete,/Ricardo-arjona-adios-melancolia-lyrics,"{'api_path': '/artists/358670', 'header_image_...",NaN,https://images.genius.com/12b6da30eea50fdbcd88...,"{'hot': False, 'unreviewed_annotations': 0}",Adios Melancolia,Adios Melancolia,https://genius.com/Ricardo-arjona-adios-melanc...


In [40]:
del df['annotation_count']
del df['api_path']
del df['full_title']
del df['header_image_thumbnail_url']
del df['header_image_url']
del df['lyrics_owner_id']
del df['path']
del df['primary_artist']
del df['pyongs_count']
del df['song_art_image_thumbnail_url']
del df['stats']
#wow, almost all information is useless
del df['title_with_featured']
del df['lyrics_state']

In [41]:
df.head()

,id,lyrics,title,url
0,1833971,\n\nEl misterio estaba allí\nEn aquella casa c...,Abarroteria de amor,https://genius.com/Ricardo-arjona-abarroteria-...
1,985362,\n\nEl azar es la metáfora perfecta\nDe quien ...,A cara o cruz,https://genius.com/Ricardo-arjona-a-cara-o-cru...
2,1141487,\n\nAcompáñame a estar solo\nA purgarme los fa...,Acompáñame a Estar Solo,https://genius.com/Ricardo-arjona-acompaname-a...
3,2123441,\n\nAcompáñame a estar solo\nA purgarme los fa...,Acompñame a estar solo - en vivo,https://genius.com/Ricardo-arjona-acompname-a-...
4,1567279,"\n\nTengo un domingo en stand by, por si algÃº...",Adios Melancolia,https://genius.com/Ricardo-arjona-adios-melanc...


#### Removes excess newlines

In [42]:
df['lyrics'].replace(regex=True,inplace=True,to_replace=r'(\n)+',value=r'\n')
df['lyrics'].replace(regex=True,inplace=True,to_replace=r'^(\n)+',value=r'')

In [43]:
df.to_csv('data/arjona_raw_songs_v2.csv', sep=';',index=False)

#### Some manual corrections of lyrics

In [44]:
#df = pd.read_csv('data/arjona_raw_songs_v2.csv', sep=';')
#df.head()


,id,lyrics,title,url
0,1833971,El misterio estaba allí\nEn aquella casa color...,Abarroteria de amor,https://genius.com/Ricardo-arjona-abarroteria-...
1,985362,El azar es la metáfora perfecta\nDe quien le b...,A cara o cruz,https://genius.com/Ricardo-arjona-a-cara-o-cru...
2,1141487,Acompáñame a estar solo\nA purgarme los fantas...,Acompáñame a Estar Solo,https://genius.com/Ricardo-arjona-acompaname-a...
3,2123441,Acompáñame a estar solo\nA purgarme los fantas...,Acompñame a estar solo - en vivo,https://genius.com/Ricardo-arjona-acompname-a-...
4,1567279,"Tengo un domingo en stand by, por si algÃºn lu...",Adios Melancolia,https://genius.com/Ricardo-arjona-adios-melanc...


In [ ]:
# df = pd.read_csv('data/raw_songs_v3.tsv', sep='\t')
# df

In [46]:
df.nunique(), df.shape

(id        256
 lyrics    238
 title     256
 url       256
 dtype: int64, (256, 4))

In [48]:
df[df.duplicated(subset=['lyrics'])]

,id,lyrics,title,url
20,1589331,Donde estes\nCreo que es justo que te enteres\...,Aún te amo (Carta No. 1),https://genius.com/Ricardo-arjona-aun-te-amo-c...
26,1951427,Yo no vine a recordar el invierno\nNi a decirl...,Cada quien su invierno - live,https://genius.com/Ricardo-arjona-cada-quien-s...
46,2116985,¿Cuando fue la ultima vez que viste las estrel...,Cundo - versión pop,https://genius.com/Ricardo-arjona-cundo-versio...
53,1932605,No es ninguna aberracion sexual\nPero me gusta...,Desnuda - en vivo,https://genius.com/Ricardo-arjona-desnuda-en-v...
89,1970600,Eran las diez de la noche\nPiloteaba mi nave\n...,Historia - en vivo,https://genius.com/Ricardo-arjona-historia-en-...
101,1953824,La nana la despertó a las seis con cuarenta y ...,La Nena (Bit‡cora de un Secuestro),https://genius.com/Ricardo-arjona-la-nena-bitc...
154,1642317,Pensar en ti\nAllá afuera seis contra seis en ...,Pensar en ti (versión acústica),https://genius.com/Ricardo-arjona-pensar-en-ti...
180,1454013,Deme un Tylenol pa'l mal de cuerpo\nY un Diaze...,Receta (versión acústica),https://genius.com/Ricardo-arjona-receta-versi...
193,1211278,"Se nos muere el amor, tiene fiebre de frio\nSe...",Se nos muere el amor,https://genius.com/Ricardo-arjona-se-nos-muere...
195,2114768,Se nos muere el amor\nTiene hambre de frio\nSe...,Se nos muere el amor - new version,https://genius.com/Ricardo-arjona-se-nos-muere...


In [50]:
indexes_to_drop = df[df.duplicated(subset=['lyrics'])].index

In [51]:
df = df.drop(indexes_to_drop).reset_index(drop=True)

In [52]:
df.drop(['url', 'id'], axis=1)

,lyrics,title
0,El misterio estaba allí\nEn aquella casa color...,Abarroteria de amor
1,El azar es la metáfora perfecta\nDe quien le b...,A cara o cruz
2,Acompáñame a estar solo\nA purgarme los fantas...,Acompáñame a Estar Solo
3,Acompáñame a estar solo\nA purgarme los fantas...,Acompñame a estar solo - en vivo
4,"Tengo un domingo en stand by, por si algÃºn lu...",Adios Melancolia
5,Me despierto y pedaleo\nSoy ciclista en este v...,A la Luna en Bicicleta
6,A la medida de mis temores\nDe mis defectos y ...,A la medida - versión demo
7,Dibujando caracolas en la luna\nCanalizo la fo...,Aleluya
8,"Amarte a ti no es lo mejor, lo tengo claro\nHa...",Amarte A Ti
9,Amo la costura de tu falda\nAmo la estrucuta d...,Amor de Tele


In [53]:
df.to_csv('data/arjona_raw_songs_v3.csv', sep=';',index=False)